In [114]:
from pyspark import SparkContext, SparkConf

import json

conf = SparkConf()
# create Spark context with necessary configuration
sc = SparkContext.getOrCreate(conf=conf)

stopwords = ['they','she','he','it','the','as','is','and']
targetList = ['.',',',';','?','!','(',')','\n','\r','"','  ',':','[',']','\t','|','-',"'s"]

def replaceAndSplit(word_path): 
    filename = word_path[0]
    content = word_path[1].lower()
    for c in targetList:
        content = content.replace(c, ' ')
    words = content.split();
    res = []
    for word in words :
        res.append((word,filename))
    
    return res


file_path = '/notebooks/Data/*/*'

rdd_whole_file = sc.wholeTextFiles(file_path) # filename + content

rdd_whole_word = rdd_whole_file.flatMap(replaceAndSplit) # (word , filename)

rdd_remove_stopword = rdd_whole_word.filter(lambda whole_word : whole_word[0] not in stopwords)

rdd_wordCount = rdd_remove_stopword.map(lambda word: ((word[0],word[1]),1)).reduceByKey(lambda a,b:a +b)# ((word , filename) , count)

rdd_wordCountAll = rdd_wordCount.map(lambda word: ((word[0][0], (word[0][1],word[1])))).reduceByKey(lambda a,b:a +b)# (word , (filename ,count))

rdd_res = rdd_wordCountAll.map(lambda word: ((word[0], [word[1]])))

rdd_res.saveAsTextFile("/notebooks/output")



